In [225]:
import sys
import scipy.io
import numpy
import math
import csv
from operator import add

from gurobipy import *

# set the directory path
import os
folder_name = os.getcwd()

epsilon = 10e-6

import time

In [226]:
# The function returns a tuple: The first element is the set B; 
#                               The second is the number of each shape in B
def getB():
    n_frequency = [ num_fq_unit - num_shape_fq_unit[i] + 1 for i in range(4) ]
    n_time = [ num_tm_unit - num_shape_tm_unit[i] + 1 for i in range(4) ]
    n = [ int(a*b) for a,b in zip(n_frequency,n_time) ]
    
    B1 = range(n[0]) # indices of shape 1
    B2 = range(n[0],n[0]+n[1]) # indices of shape 2
    B3 = range(n[0]+n[1],n[0]+n[1]+n[2]) # indices of shape 3 
    B4 = range(n[0]+n[1]+n[2],n[0]+n[1]+n[2]+n[3]) # indices of shape 4

    B = B1 + B2 + B3 + B4
    return [B, n]

In [227]:
time_length = 2*1e-3 # in s
frequency_length = 2000 # in kHz

subcarrier_bw = 15 # subcarrier bandwidth (kHz)
unit_tl = 0.125*1e-3 # in s
unit_fl = 15*12 # in kHz

num_fq_unit = int(frequency_length/unit_fl)
num_tm_unit = int(time_length/unit_tl)

num_latency = 5 # the number of services constrained by latency
num_capacity = 10-num_latency # the number of services to maximize capacity
num_service = num_latency + num_capacity # total number of services

tti = [ x * 1e-3 for x in [0.5, 0.25, 0.125, 0.125] ]
subcarrier_num = [ 12, 24, 48, 48 ] # one subcarrier is 15 kHz
shape_bw = [ n*subcarrier_bw for n in subcarrier_num ]
num_shape_fq_unit = [ int(bw/unit_fl) for bw in shape_bw ]
num_shape_tm_unit = [ int(tl/unit_tl) for tl in tti ] 

tau = [ t*time_length for t in [1.0, 0.7, 0.8, 0.75, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] ]

tmp = getB()
B_entire = tmp[0]
nB = tmp[1]

In [228]:
# # set of physical layer blocks (PRBs)
with open('B.csv', 'rb') as f:
    B_csv = csv.reader(f)
    B_entire = list(B_csv)
    B_entire = [item for sublist in B_entire for item in sublist] # flatten list
    B_entire = map(int, map(float, B_entire)) # convert to int
    
# # set of latency services    
with open('Kl.csv', 'rb') as f:
    Kl_csv = csv.reader(f)
    Kl = list(Kl_csv)
    Kl = [item for sublist in Kl for item in sublist] # flatten list
    Kl = map(int, map(float, Kl)) # convert to int
    
# # set of capacity services    
with open('Kc.csv', 'rb') as f:
    Kc_csv = csv.reader(f)
    Kc = list(Kc_csv)
    Kc = [item for sublist in Kc for item in sublist] # flatten list
    Kc = map(int, map(float, Kc)) # convert to int
    
# # set of all services
K = Kl + Kc

# # set of resource units (RUs)
with open('I.csv', 'rb') as f:
    I_csv = csv.reader(f)
    I = list(I_csv)
    I = [item for sublist in I for item in sublist] # flatten list
    I = map(int, map(float, I)) # convert to int

In [229]:
# # matrix a
with open('a.csv', 'rb') as f:
    a_csv = csv.reader(f)
    a = list(a_csv)
    a = [ map(int,map(float,x)) for x in a] # convert to int

In [230]:
# if conflict_PRBs[b1][b2] == True, then the PRBs b1 b2 cannot be used simultaneously
with open('conflict_PRB.csv', 'rb') as f:
    conflict_PRB_csv = csv.reader(f)
    conflict_PRB = list(conflict_PRB_csv)
    conflict_PRB = [ map(lambda y: int(float(y))==1, x) for x in conflict_PRB]

In [231]:
with open('DataVol_list.csv', 'rb') as f:
    datavol_csv = csv.reader(f)
    datavol_table = list(datavol_csv)
    datavol_table = [ map(float,x) for x in datavol_table] # convert to float
    datavol_table = map(list, zip(*datavol_table)) # transpose table

from scipy.interpolate import interp1d

# interplotation for datavol_table
x = range(1,31)
y1 = datavol_table[0] # for shape 1
y2 = datavol_table[1] # for shape 2
y3 = datavol_table[2] # for shape 3
y4 = datavol_table[3] # for shape 4

datavol_interpld = [interp1d(x,y1), interp1d(x,y2), interp1d(x,y3), interp1d(x,y4)]

In [232]:
def shapeCapacity(PRB_shape, avg_snr): 
    if PRB_shape < 0 or PRB_shape > 3:
        print 'shape_index is out of range: valid index is between 0--3'
        return
    return datavol_interpld[PRB_shape](avg_snr)  # capacity is in bit

In [233]:
def getPRBShape(PRB_index):
    if PRB_index < nB[0]:
        PRB_shape = 0
    elif PRB_index < nB[0]+nB[1]:
        PRB_shape = 1
    elif PRB_index < nB[0]+nB[1]+nB[2]:
        PRB_shape = 2
    else:
        PRB_shape = 3
    return PRB_shape

In [234]:
def getr(snr_mat, B):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in B:
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [235]:
def getrShape0(snr_mat, B):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in range(nB[0]):
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [236]:
def getrShape1(snr_mat, B):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in range(nB[1]):
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [237]:
def getrShape2(snr_mat, B):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in range(nB[2]):
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [238]:
def getAlpha(lam, r, q, B):
    alpha=[]
    for b in B:
        alpha_b = sum( lam[i]*a[b][i] for i in I ) 
        alpha.append(alpha_b)
    return alpha

In [239]:
# The argument lam is lambda
# The function returns matrix x, with only columns in Kc being computed.
# The columns in Kl of the returned matrix x are zero vectors.
def solveP2( lam, r, q, B):
    PRB_alloc = [ -1 for b in B ] # PRB_alloc[b] is the index of service that PRB b should be allocated to
                                  # PRB_alloc[b]=-1 means that PRB b is not allocated
        
    sol_x = [ [ 0 for k in K ] for b in B ] # variables to be returned
    
    alpha = getAlpha(lam,r, q, B)
    for b in B:
        tmp_list = [ r[b][k]-alpha[b] for k in Kc ]
        if max(tmp_list) > 0:   # PRB is allocated only if r[b][k]-alpha[b] is positive
            PRB_alloc[b] = len(Kl) + numpy.argmax(tmp_list) # Kl is added such that the value of PRB_alloc[b] is 
                                                       # coherent with the corresponding indexed position in K
    
    # Convert PRB_alloc to matrix x
    for b in B:
        k = PRB_alloc[b] # indexed service
        if k >= 0: # indicating that PRB_alloc[b] != -1
            sol_x[b][k] = 1
    
    return sol_x

In [240]:
# The argument lam is lambda
# The function returns matrix x, with only columns in Kl being computed.
# The columns in Kc of the returned matrix x are zero vectors.
def solveP3( lam, r, q, B):
    # create optimization model
    modelKl = Model('Integer Programming - Kl')
    modelKl.modelSense = GRB.MINIMIZE
    modelKl.setParam('OutputFlag', False) # slience output
    
    # create varialbes for modelKl:
    xKl = []
    for b in B:
        xKl_b = []
        for k in Kl:
            xKl_b.append(modelKl.addVar(vtype=GRB.BINARY))
        xKl.append(xKl_b)
    modelKl.update()
    
    # add constraints 
    for k in Kl:
        modelKl.addConstr( sum(r[b][k]*xKl[b][k] for b in B) >= q[k] )
    modelKl.update()

    # set objective function
    alpha = getAlpha(lam,r,q,B)
    modelKl.setObjective(
        sum( alpha[b]*xKl[b][k] for k in Kl for b in B )
    )
    
    # solve modelKl
    modelKl.optimize()
    
    # construct variables to be returned 
    sol_x = [ [ 0 for k in K ] for b in B ]
    if modelKl.Status == GRB.OPTIMAL or modelKl.Status == GRB.SUBOPTIMAL:
        for b in B:
            for k in Kl:
                sol_x[b][k] = int(xKl[b][k].x)
            
    return sol_x

In [241]:
def assignBlockLD(lam, x_count, r, q, B):
    priority = [ [i[0] for i in sorted(enumerate(x_count[k]), key=lambda y:y[1],reverse=True)] for k in K ]
    
    best = -1
    sol_x_best = []
    for i in range(400):
        sol_x = [ [ 0 for k in K ] for b in B ] # variables to be returned
        collision = [ False for b in B ]

        Kl_rand = numpy.random.permutation(Kl)
        Kc_rand = numpy.random.permutation(Kc)
        B_rand = numpy.random.permutation(B)

        for k in Kl_rand:
            service_bit = sum(r[b][k]*sol_x[b][k] for b in B)
            while service_bit < q[k] and collision.count(False)>0: 
                for pos in range(len(priority[k])):
                    b = priority[k][pos]
                    if collision[b] == False: 
                        sol_x[b][k] = 1
                        service_bit += r[b][k]*sol_x[b][k]
                        for p in B: # set all PRB overlapping with b to be in collision
                            if conflict_PRB[b][p] == True:
                                collision[p] = True
                        break

        alpha = getAlpha(lam,r,q, B)
        for b in B_rand:
            alloc = True
            if collision[b]==True: # b wouldn't be allocated if in collision
                alloc = False 
            if alloc == True: 
                tmp_list = [ r[b][k]-alpha[b] for k in Kc ]
                user_to_alloc = len(Kl) + numpy.argmax(tmp_list) 
                sol_x[b][user_to_alloc] = 1
                for p in B: # set all PRB overlapping with b to be in collision
                    if conflict_PRB[b][p] == True:
                        collision[p] = True
                        
        curr = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B)
        if curr > best:
            best = curr
            sol_x_best = sol_x
                    
    return sol_x_best

In [242]:
def assignBlockLP(x_count, r, q, B):
    sol_x = [ [ 0 for k in K ] for b in B ]
    collision = [ False for b in B ]
    priority = [ [i[0] for i in sorted(enumerate(x_count[k]), key=lambda y:y[1],reverse=True)] for k in K ]
    best_sol = [];
    
    best = -1
    for k in range(400):
        Kl_rand = numpy.random.permutation(Kl)
        Kc_rand = numpy.random.permutation(Kc)
        B_rand = numpy.random.permutation(B)

        for k in Kl_rand:
            service_bit = sum(r[b][k]*sol_x[b][k] for b in B)
            while service_bit < q[k] and collision.count(False)>0: 
                for pos in range(len(priority[k])):
                    b = priority[k][pos]
                    if collision[b] == False: 
                        sol_x[b][k] = 1
                        service_bit += r[b][k]*sol_x[b][k]
                        for p in B: # set all PRB overlapping with b to be in collision
                            if conflict_PRB[b][p] == True:
                                collision[p] = True
                        break
                
        for k in Kc_rand:
            for b in B_rand:
                if collision[b] == False and x_count[k][b] >= 0.5:
                    sol_x[b][k]=1
                    for p in B:
                        if conflict_PRB[b][p] == True:
                            collision[p] = True

        while collision.count(False)>0:
            for k in Kc_rand:
                for pos in range(len(priority[k])):
                    b = priority[k][pos]
                    if collision[b] == False:
                        sol_x[b][k] = 1
                        for p in B:
                            if conflict_PRB[b][p] == True:
                                collision[p] = True
                        break

        obj = sum( r[b][k]*int(sol_x[b][k]) for k in Kc for b in B )
        if obj > best:
            best = obj
            best_sol = sol_x

    return best_sol

In [243]:
def isFeasible(x, r, q, B):
    if (numpy.dot( numpy.dot(numpy.matrix(a).transpose(), 
                             numpy.matrix(x)), numpy.ones(len(K)) ) 
        > numpy.ones(len(I))).tolist()[0].count(True)>0:
        return False
    if (numpy.dot(numpy.multiply(numpy.matrix(r),numpy.matrix(x))[:,0:len(Kl)].transpose(),
                  numpy.ones(len(B)))<numpy.matrix(q)).tolist()[0].count(True)>0:
        return False
    return True

In [244]:
def LPBased(rho, r, q, B):
    model = Model('Linear Programming')
    model.modelSense = GRB.MAXIMIZE
    model.setParam('OutputFlag', False) # slience output

    x = []
    for b in B:
        x_b = []
        for k in K:
            x_b.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1))
        x.append(x_b)

    model.update()

    for k in Kl:
        model.addConstr( sum(r[b][k]*x[b][k] for b in B) >= q[k] )
    model.update()

    for i in I:
        model.addConstr( sum(a[b][i]*x[b][k] for k in K for b in B ) <= 1 )
    model.update()

    model.setObjective(
        sum( r[b][k]*x[b][k] for k in Kc for b in B )
    )

    model.optimize()
    
    x_count = [ [ 0 for b in B] for k in K ]
    if model.Status == GRB.OPTIMAL or model.Status == GRB.SUBOPTIMAL:
        x_count = [ [ x[b][k].x for b in B] for k in K ]
        for b in B:
            for k in K:
                if x[b][k].x < rho:
                    x_count[k][b] = 0
    
    return x_count

In [245]:
def LDBased(M, varphi, beta, gamma, r, q, B):
    x_count_LP = LPBased(0,r,q,B)
    x_LP = assignBlockLP(x_count_LP,r,q,B)
    obj_LP =  sum(r[b][k]*x_LP[b][k] for k in Kc for b in B)
    
    lam = [1 for i in I]
    x_prev = [ [0 for k in K ] for b in B ]
    penalty = [ 1 for i in I ]
    x_count = [ [ 0 for b in B ] for k in K ]
    best_dual_sofar = 1e10
    eta = 0.95
    no_improve_count = 0
    
    for j in range(1, M+1): # k belongs to [1,100]
        
        xKc = solveP2(lam,r,q,B)
        xKl = solveP3(lam,r,q,B)
        x = (numpy.matrix(xKl) + numpy.matrix(xKc)).tolist() 

        # obtain the corresponding dual function value under current lambda

        dual = sum(r[b][k]*x[b][k] for k in Kc for b in B) + sum( lam[i]*(1-sum(a[b][i]*x[b][k] for k in K for b in B)) for i in I)
        if dual < best_dual_sofar + 0.005:
            no_improve_count = 0
            best_dual_sofar = dual
            gamma = min(gamma*1.05, 1.9)
        else:
            no_improve_count += 1

        if no_improve_count >=1:
            gamma = 0.95*gamma
            no_improve_count = 0

        numurator = abs(dual - obj_LP*beta)
        denominator = numpy.linalg.norm(penalty, 2)**2

        if j<varphi:
            mu = 10/float(j**0.5)
        else:
            mu = gamma*numurator/denominator

        penalty = (numpy.ones(len(I))- numpy.dot( numpy.dot(numpy.matrix(a).transpose(), numpy.matrix(x)), numpy.ones(len(K)))).tolist()[0]
        lam = map(lambda y:max(y,0), (numpy.matrix(lam) - mu*numpy.matrix(penalty)).tolist()[0])

        x_count = (numpy.matrix(x_count) + numpy.matrix(x).transpose()).tolist()    
        
#         print 'iteration', j, 'step length=', mu, 'dual=', best_dual_sofar, 'gamma=', gamma
        
#     print 'LD upper bound = ', best_dual_sofar
#     print 'Solving LD costs', time.time()-start_time, 'seconds'
    return [lam, x_count, best_dual_sofar]

In [246]:
def optSolver(r, q, B):
    model = Model('Interger Programming')
    model.modelSense = GRB.MAXIMIZE
    model.setParam('OutputFlag', False) # slience output
    model.setParam('TimeLimit', 3*60) # time_limit for gurobi
    
    sol_x = [ [ 0 for k in K ] for b in B ]

    x = []
    for b in B:
        x_b = []
        for k in K:
    #         x_b.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1))
            x_b.append(model.addVar(vtype=GRB.BINARY))
        x.append(x_b)

    model.update()

    for k in Kl:
        model.addConstr( sum(r[b][k]*x[b][k] for b in B) >= q[k] )
    model.update()

    for i in I:
        model.addConstr( sum(a[b][i]*x[b][k] for k in K for b in B ) <= 1 )
    model.update()

    model.setObjective(
        sum( r[b][k]*x[b][k] for k in Kc for b in B )
    )

    model.optimize()
    
    if model.Status == GRB.OPTIMAL or model.Status == GRB.SUBOPTIMAL:
        for b in B:
            for k in K:
                sol_x[b][k] = x[b][k].x
            
    return sol_x

In [247]:
# import random
# sample_num = 40

# snr_mat_list = [ numpy.matrix( [ [random.uniform(5,30) for i in K ] for u in range(11) ] ) for h in range(sample_num) ] 
# B_inuse = B_entire

# for normalized_rate in [0.032，0.064，0.128，0.256，0.512，1.024]:
    
#     for h in range(sample_num):
        
#         start_time = time.time()
        
#         snr_mat = snr_mat_list[h]
#         curr_r = getr(snr_mat, B_inuse)
#         curr_q = [ q_elem * 1024 * normalized_rate for q_elem in [ 1, 1, 1, 1, 1 ] ] # Kbit
        
#         x_opt = optSolver(r=curr_r, q=curr_q, B=B_inuse)
#         obj_opt = sum(curr_r[b][k]*x_opt[b][k] for k in Kc for b in B_inuse)
        
#         y_LP = LPBased(0, r=curr_r, q=curr_q, B=B_inuse)
#         x_LP = assignBlockLP(y_LP, r=curr_r, q=curr_q, B=B_inuse)
#         obj_LP = sum(curr_r[b][k]*x_LP[b][k] for k in Kc for b in B_inuse)
#         bound_LP = sum(curr_r[b][k]*y_LP[k][b] for k in Kc for b in B_inuse)
        
#         result = LDBased(M=200, varphi=5, beta=0.9, gamma=0.5, r=curr_r, q=curr_q, B=B_inuse)  
#         lam = result[0]
#         y_LD = result[1]
#         x_LD = assignBlockLD(lam, y_LD, r=curr_r ,q=curr_q, B=B_inuse)
#         obj_LD = sum(curr_r[b][k]*x_LD[b][k] for k in Kc for b in B_inuse)
#         bound_LD = result[2]
        
#         bound = min(bound_LP,bound_LD)
#         obj = max(obj_LP,obj_LD)
        
#         obj_gap_smp = abs(obj_opt - obj)/obj_opt
#         obj_LP_gap_smp = abs(obj_opt - obj_LP)/obj_opt
#         obj_LD_gap_smp = abs(obj_opt - obj_LD)/obj_opt
        
#         bound_gap_smp = abs(bound - obj_opt)/obj_opt
#         bound_LP_gap_smp = abs(bound_LP - obj_opt)/obj_opt
#         bound_LD_gap_smp = abs(bound_LD - obj_opt)/obj_opt 
    
#         print 'rate = ', normalized_rate, 'obj_gap_smp = ', obj_gap_smp, 'obj_LP_gap_smp = ', obj_LP_gap_smp, 'obj_LD_gap_smp = ', obj_LD_gap_smp, 'bound_gap_smp = ', bound_gap_smp, 'bound_LP_gap_smp = ', bound_LP_gap_smp, 'bound_LD_gap_smp = ', bound_LD_gap_smp
        
        

In [248]:
snr_mat = numpy.matrix([[ 24.42130695,  27.01265871,  29.30201528,   7.13861208,
          16.47898114,  22.69447115,  21.74435108,  15.42748504,
          23.56379969,  26.33527111],
        [ 22.33190727,  26.99235666,   7.7562922 ,  10.54346664,
          15.52839329,  26.74788932,  27.38548404,  14.48918902,
          24.63719871,  19.63629382],
        [ 17.51724531,  23.37919956,  26.98579328,   9.35025715,
          22.32692102,  11.93421716,  10.23993204,  15.55815635,
          12.89071123,   9.24137262],
        [ 29.54918125,  12.07632449,  24.01538095,  28.15479741,
          29.75195359,  10.190799  ,  14.2045514 ,  17.49368559,
           9.88447589,   9.73392575],
        [ 25.0223455 ,  23.90227576,  17.44670836,   6.18379495,
          22.84289628,  16.09700852,  16.30535449,  10.56226018,
           9.46042129,  13.6695326 ],
        [ 21.11313716,   8.86805933,  19.54751904,   9.56340909,
          16.1966827 ,  15.93872269,  10.0835697 ,  13.92085506,
          20.56351892,  22.99725646],
        [ 28.40718882,  29.98014357,  28.94975632,  21.87469362,
          29.07119697,   9.87290022,   5.62181866,  12.02532803,
          12.41564751,  24.0018508 ],
        [ 12.06698104,  14.97052524,  26.20469541,  27.99274488,
          12.46333221,  19.99327262,  14.13906995,   5.9604763 ,
          19.18106013,  19.13080749],
        [ 15.64031319,  10.2268432 ,  15.02498064,  22.47818857,
          18.82352462,  29.82238321,  24.00776091,  14.3763043 ,
          27.72770816,  17.20611143],
        [  5.22465262,  14.95041822,  27.85268977,  28.10794329,
           8.00642958,  23.75710242,  29.02785938,  13.28771548,
          21.3414978 ,  24.2081503 ],
        [ 18.74784454,  12.16083034,   7.18346512,  29.38097387,
          23.59674388,  23.51032958,  28.13159251,  26.51752969,
          28.08712507,   5.84124966]])

In [249]:
normalized_rate = 0.256
curr_q = [ q_elem * 1024 * normalized_rate for q_elem in [ 1, 1, 1, 1, 1 ] ]
B_inuse = B_entire

for dl in [0.125, 0.25, 0.5, 1, 1.5, 2]:
    tau = [ t*time_length for t in [dl, dl, dl, dl, dl, 1.0, 1.0, 1.0, 1.0, 1.0] ]
    
#     scalable_r = getr(snr_mat, B_inuse)
    nonscalable_r = getrShape2(snr_mat, B_inuse)
    
#     x_scalable_opt = optSolver(r=scalable_r, q=curr_q, B=B_inuse)
    x_nonscalable_opt = optSolver(r=nonscalable_r, q=curr_q, B=B_inuse)
#     obj_scalable_opt = sum(scalable_r[b][k]*x_scalable_opt[b][k] for k in Kc for b in B_inuse)
    obj_nonscalable_opt = sum(nonscalable_r[b][k]*x_nonscalable_opt[b][k] for k in Kc for b in B_inuse)  
    
#     y_scalable_LP = LPBased(0, r=scalable_r, q=curr_q, B=B_inuse)
#     x_scalable_LP = assignBlockLP(y_scalable_LP, r=scalable_r, q=curr_q, B=B_inuse)
#     obj_scalable_LP = sum(scalable_r[b][k]*x_scalable_LP[b][k] for k in Kc for b in B_inuse)
    
#     result = LDBased(M=200, varphi=5, beta=0.9, gamma=0.5, r=scalable_r, q=curr_q, B=B_inuse)  
#     lam = result[0]
#     y_scalable_LD = result[1]
#     x_scalable_LD = assignBlockLD(lam, y_scalable_LD, r=scalable_r ,q=curr_q, B=B_inuse)
#     obj_scalable_LD = sum(scalable_r[b][k]*x_scalable_LD[b][k] for k in Kc for b in B_inuse)
    
#     obj_scalable_heuristic = max(obj_scalable_LP, obj_scalable_LD)
    
#     print 'dl = ', dl, 'nonscalable_opt = ', obj_nonscalable_opt, 'scalable_heuristic = ', obj_scalable_heuristic, 'scalable_opt = ', obj_scalable_opt

    print 'dl = ', dl, 'nonscalable_opt = ', obj_nonscalable_opt, 'feasibility = ', isFeasible(x_nonscalable_opt, nonscalable_r, curr_q, B_inuse)


dl =  0.125 nonscalable_opt =  0.0 feasibility =  False
dl =  0.25 nonscalable_opt =  0.0 feasibility =  True
dl =  0.5 nonscalable_opt =  0.0 feasibility =  True
dl =  1 nonscalable_opt =  0.0 feasibility =  True
dl =  1.5 nonscalable_opt =  2188.66518186 feasibility =  True
dl =  2 nonscalable_opt =  2188.66518186 feasibility =  True


In [250]:
# print 'start...'
# import random

# normalized_rate = 0.512
# curr_q = [ q_elem * 1024 * normalized_rate for q_elem in [ 1, 1, 1, 1, 1 ] ]
# B_inuse = B_entire

# sample_num = 40
# snr_mat_list = [ numpy.matrix( [ [random.uniform(5,30) for i in K ] for u in range(11) ] ) for h in range(sample_num) ] 



# for dl in [0.25, 0.5, 0.75, 1.0]:
#     tau = [ t*time_length for t in [dl, dl, dl, dl, dl, 1.0, 1.0, 1.0, 1.0, 1.0] ]
    
#     for h in range(sample_num):
    
#         snr_mat = snr_mat_list[h]

#         scalable_r = getr(snr_mat, B_inuse)
#         nonscalable_r = getrShape0(snr_mat, B_inuse)

#         x_scalable_opt = optSolver(r=scalable_r, q=curr_q, B=B_inuse)
#         x_nonscalable_opt = optSolver(r=nonscalable_r, q=curr_q, B=B_inuse)
#         obj_scalable_opt = sum(scalable_r[b][k]*x_scalable_opt[b][k] for k in Kc for b in B_inuse)
#         obj_nonscalable_opt = sum(nonscalable_r[b][k]*x_nonscalable_opt[b][k] for k in Kc for b in B_inuse)  

#         y_scalable_LP = LPBased(0, r=scalable_r, q=curr_q, B=B_inuse)
#         x_scalable_LP = assignBlockLP(y_scalable_LP, r=scalable_r, q=curr_q, B=B_inuse)
#         obj_scalable_LP = sum(scalable_r[b][k]*x_scalable_LP[b][k] for k in Kc for b in B_inuse)

#         result = LDBased(M=200, varphi=5, beta=0.9, gamma=0.5, r=scalable_r, q=curr_q, B=B_inuse)  
#         lam = result[0]
#         y_scalable_LD = result[1]
#         x_scalable_LD = assignBlockLD(lam, y_scalable_LD, r=scalable_r ,q=curr_q, B=B_inuse)
#         obj_scalable_LD = sum(scalable_r[b][k]*x_scalable_LD[b][k] for k in Kc for b in B_inuse)

#         obj_scalable_heuristic = max(obj_scalable_LP, obj_scalable_LD)
        
#         fb_nonscalable_opt = isFeasible(x_nonscalable_opt, nonscalable_r, curr_q, B_inuse)
#         fb_scalable_heuristic = isFeasible(x_scalable_LP, scalable_r, curr_q, B_inuse) or isFeasible(x_scalable_LD, scalable_r, curr_q, B_inuse)
#         fb_scalable_opt = isFeasible(x_scalable_opt, scalable_r, curr_q, B_inuse)

#         print 'dl = ', dl, obj_nonscalable_opt, obj_scalable_heuristic, obj_scalable_opt, fb_nonscalable_opt, fb_scalable_heuristic, fb_scalable_opt
